In [1]:
import datetime
import dateutil.parser
import pprint
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
# Strasbourg, Schiltigheim, Illkirch-Graffenstaden
valid_cp = ['67000', '67100', '67200', '67300', '67400']

In [3]:
dict_adresse = dict()
dict_nom = dict()
dict_adresse_reverse = dict()
dict_nom_reverse = dict()

In [4]:
# Create csv output
with open('raw/prix-carburants.csv', 'w', encoding='utf-8') as file:
    # csv columns
    file.write('pdv_id,carburant_id,date,carburant_prix\n')
    # for each year we have data
    for annee in range(2008, 2023):
        # parse xml data
        data_xml = ET.parse(f'raw/PrixCarburants_annuel_2007-2022/PrixCarburants_annuel_{annee}.xml')
        # for each seller (point de vente)
        for pdv in data_xml.getroot():
            # check cp is in valid_cp (code postale)
            if pdv.tag == 'pdv' and pdv.attrib['cp'] in valid_cp:
                # seller id
                pdv_id = int(pdv.attrib['id'])
                pdv_adresse = ''
                for data in pdv:
                    # address entry
                    if data.tag == 'adresse':
                        pdv_adresse = data.text
                        # dict of address
                        dict_adresse[pdv_id] = pdv_adresse
                        dict_adresse_reverse[pdv_adresse] = pdv_id
                    # price entry
                    if data.tag == 'prix':
                        try:
                            data_nom  = data.attrib['nom']
                            data_id   = int(data.attrib['id'])
                            data_date = dateutil.parser.parse(data.attrib['maj']).strftime("%d/%m/%Y %H:%M")
                            data_val  = float(data.attrib['valeur'])
                            
                            if data_val > 10:
                                data_val /= 1000
                            
                            # dict of data names
                            dict_nom[data_id] = data_nom
                            dict_nom_reverse[data_nom] = data_id
                            # write data out
                            file.write(f'{pdv_id},{data_id},{data_date},{data_val}\n')
                        except Exception as e:
                            pass

In [5]:
pprint.pprint(dict_adresse)

{67000002: 'Route de la Wantzenau',
 67000003: '49 Route du Rhin',
 67000004: '82 Rue du Faubourg-National',
 67000005: '41 Boulevard de Lyon',
 67000006: '50 Boulevard de la Victoire',
 67000007: 'CENTRE ROUTIER DE RHEINFELD',
 67000009: '10 Rue de Rome',
 67000010: '169 Route de Schirmeck',
 67000011: "9 Avenue d'Alsace",
 67000012: '59 Rue Boecklin',
 67000013: '57 Rue de Ribeauvillé',
 67000015: 'rue saint léon',
 67000016: '1 rue saint léon',
 67000017: '1 rue Saint Léon',
 67000018: '1 rue Saint Léon',
 67000019: '1 Rue Saint-Léon',
 67000020: '1 rue saint léon',
 67000022: '6 Rue du Rheinfeld',
 67000023: '1 rue saint léon',
 67000024: 'TEST',
 67000025: '50 BOULEVARD DE LA VICTOIRE',
 67000026: '49 RTE DU RHIN',
 67000027: 'CENTRE ROUTIER DE RHEINFELD',
 67000028: '59 RUE BOECKLIN',
 67033001: 'place André Maurois',
 67100002: '250 Avenue de Colmar',
 67100003: "4 Route d'Altenheim",
 67100005: '212 Avenue de Colmar',
 67100007: '10 Rue du Landsberg',
 67100008: 'RUE DE RHEINFE

In [6]:
pprint.pprint(dict_adresse_reverse)

{'1 Rue Saint-Léon': 67000019,
 '1 rue Saint Léon': 67000018,
 '1 rue saint léon': 67000023,
 '10 Rue de Rome': 67000009,
 '10 Rue du Landsberg': 67100007,
 "142 ROUTE D' OBERHAUSBERGEN": 67200012,
 '142 Route d#039;Oberhausbergen': 67200004,
 '145 ROUTE DE LYON': 67400005,
 '145 Route de Lyon': 67400003,
 '169 Route de Schirmeck': 67000010,
 '212 Avenue de Colmar': 67100005,
 '250 Avenue de Colmar': 67100002,
 '256 route de Mittelhausbergen': 67205001,
 '256, route de Mittelhausbergen': 67205001,
 '284 ROUTE DE SCHIRMECK': 67200001,
 '31 Route de Mittelhausbergen': 67200008,
 '38 Route de Lyon': 67400002,
 "4 ROUTE D'ALTENHEIM": 67100009,
 "4 Route d'Altenheim": 67100003,
 '41 Boulevard de Lyon': 67000005,
 '48 Route des Romains': 67200005,
 '48, avenue des Romaines': 67200009,
 '48-58 route des romains': 67200010,
 '49 RTE DU RHIN': 67000026,
 '49 Route du Rhin': 67000003,
 '50 BOULEVARD DE LA VICTOIRE': 67000025,
 '50 Boulevard de la Victoire': 67000006,
 '57 Rue de Ribeauvillé': 67

In [7]:
pprint.pprint(dict_nom)
pprint.pprint(dict_nom_reverse)

{1: 'Gazole', 2: 'SP95', 3: 'E85', 4: 'GPLc', 5: 'E10', 6: 'SP98'}
{'E10': 5, 'E85': 3, 'GPLc': 4, 'Gazole': 1, 'SP95': 2, 'SP98': 6}
